In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from function_util import *
from IMP import IMP
from SMP1 import SMP1
from SMP2 import SMP2
from FIS1 import FIS1
from FIS2 import FIS2
from FIS3 import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load all the data 
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')

# select error bound 
err_size = 38
# select cut off point for smp1, for use 60 following Quddus paper
cutoff = 60
# select number of no match needed to break iteration 
no_match_max = 3

# weight perpendicular distance more 
wt_matrix = build_matrix3(rule_3 = 1.5, rule_8 = 1.2)

# intialize no match counter 
no_match = 0 
# initialize matched trajectory list
matched_point_list = []
# initialize matched edge list 
matched_edge_list = []
# initialize SMP1 result list 
res_list =[]
# initialize counter for matched_list 
count = 0

point_index = 0

# Run IMP Algorithm to find first point 
point_index, curr_edge = IMP(point_index, gdf_utm, edges_utm, 7)

curr_edge['method'] = ['IMP']
#store current edge 
matched_edge_list.append(curr_edge)

# matched current location with current edge 
curr_loc = gdf_utm.iloc[[point_index]]
matched_point = point_matching(curr_loc, curr_edge.iloc[0])
# find longitude and latitude for last matched data
matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
matched_point['method'] = ['IMP']
# update previous matched position
last_matched = matched_point

# store matched point 
matched_point_list.append(matched_point)

# move to the next iteration with for loop
start_iter = point_index + 1


for point_index in range(start_iter, len(gdf_utm)):    
    # update last location 
    prev_loc = gdf_utm.iloc[[point_index - 1]]

    # update current location 
    curr_loc = gdf_utm.iloc[[point_index]]

    # perform SMP1 
    res = SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm)
    # store result list
    res_list.append(res)

    # decide whether vehicle is entering junction or not cut off 
    if res >= cutoff and res < 99:
        # vehicle is not entering a new edge, matched edge doesnt change
        # update matched edge list 
        print(['entering SMP1 at iteration'], point_index)
        curr_edge['method'] = ['SMP1']
        matched_edge_list.append(curr_edge)
        # update matched point
        matched_point = point_matching(curr_loc, curr_edge.iloc[0])
        # find longitude and latitude for last matched data
        matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
        matched_point['method'] = ['SMP1']
        # store matched point 
        matched_point_list.append(matched_point)
        # update previous matched position
        last_matched = matched_point
    elif res < cutoff:
        prev_edge = curr_edge
        # SMP1 detect vehicle is entering new edge, perform SMP 2 to update current edge
        edge_candidate = SMP2(curr_loc, curr_edge, prev_loc, last_matched, err_size, nodes_utm, edges_utm, gdf_utm, wt_matrix = wt_matrix)

        if edge_candidate is None:
            print(['No edge can be matched at trajectory number', point_index])
            no_match += 1
        else:
            print(['entering SMP2 at iteration'], point_index)
            curr_edge = edge_candidate
            curr_edge['method'] = ['SMP2']
            # curr_edge['prev_end_node'] = prev_edge.index[0][1]
            # there is no more matched edge, break the loop 
            matched_edge_list.append(curr_edge)
            # update matched point
            matched_point = point_matching(curr_loc, curr_edge.iloc[0])
            # find longitude and latitude for last matched data
            matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
            matched_point['method'] = ['SMP2']
            # store matched point 
            matched_point_list.append(matched_point)
            # update previous matched position
            last_matched = matched_point

    if no_match == no_match_max:
        # stop iteration when there number of no match is exceeded
        break

    # update count 
    count +=  1


['edges found at iteration number', 1]
['edges found at iteration number', 2]
['edges found at iteration number', 3]
['edges found at iteration number', 4]
['edges found at iteration number', 5]
['edges found at iteration number', 6]
['edges found at iteration number', 7]
['edges found at iteration number', 8]
['edges found at iteration number', 9]
['edges found at iteration number', 10]
['edges found at iteration number', 11]
['edges found at iteration number', 12]
['edges found at iteration number', 13]
['edges found at iteration number', 14]
['edges found at iteration number', 15]
['edges found at iteration number', 16]
['edges found at iteration number', 17]
['edges found at iteration number', 18]
['edges found at iteration number', 19]
['edges found at iteration number', 20]
['edges found at iteration number', 21]
['edges found at iteration number', 22]
['Starting SMP phase at', 21]
['entering SMP2 at iteration'] 22
['entering SMP2 at iteration'] 23
['entering SMP2 at iteration'] 

In [8]:
# create dataframe for the results
matched_edge_df = pd.concat(matched_edge_list)
matched_point_df = pd.concat(matched_point_list)

In [11]:
# measure connectivity of the link 
matched_point_df[matched_point_df['method'] == 'SMP1']

,geometry,lon_lat,method
0,POINT (516702.076 5398591.161),POINT (9.22717 48.74012),SMP1
0,POINT (516708.090 5398607.508),POINT (9.22725 48.74026),SMP1
0,POINT (516723.670 5398660.258),POINT (9.22747 48.74074),SMP1
0,POINT (516728.403 5398696.247),POINT (9.22753 48.74106),SMP1
0,POINT (516672.210 5398734.587),POINT (9.22677 48.74141),SMP1
...,...,...,...
0,POINT (518782.398 5400802.655),POINT (9.25557 48.75995),SMP1
0,POINT (518782.364 5400802.628),POINT (9.25557 48.75995),SMP1
0,POINT (518782.438 5400802.687),POINT (9.25557 48.75995),SMP1
0,POINT (518782.425 5400802.676),POINT (9.25557 48.75995),SMP1


In [42]:
np.unique(matched_edge_df['method'], return_counts=True)

(array(['IMP', 'SMP1', 'SMP2'], dtype=object),
 array([ 1, 63, 85], dtype=int64))

In [13]:
 pd.concat(matched_edge_list)

osmid   lanes     ref  \
u         v         key                                                      
279568483 287347230 0    [152021425, 231768650, 231768651]  [2, 3]  L 1200   
287347230 279568483 0    [152021425, 231768650, 231768651]  [2, 3]  L 1200   
                    0    [152021425, 231768650, 231768651]  [2, 3]  L 1200   
                    0    [152021425, 231768650, 231768651]  [2, 3]  L 1200   
                    0    [152021425, 231768650, 231768651]  [2, 3]  L 1200   
...                                                    ...     ...     ...   
373389416 373252315 0               [161583680, 243216859]  [2, 1]  L 1198   
                    0               [161583680, 243216859]  [2, 1]  L 1198   
                    0               [161583680, 243216859]  [2, 1]  L 1198   
                    0               [161583680, 243216859]  [2, 1]  L 1198   
                    0               [161583680, 243216859]  [2, 1]  L 1198   

                                        name    highway maxspeed  oneway  \
u         v         key                                                    
279568483 287347230 0     Kirchheimer Straße  secondary       60   False   
287347230 279568483 0     Kirchheimer Straße  secondary       60   False   
                    0     Kirchheimer Straße  secondary       60   False   
                    0     Kirchheimer Straße  secondary       60   False   
                    0     Kirchheimer Straße  secondary       60   False   
...                                      ...        ...      ...     ...   
373389416 373252315 0    Otto-Hirsch-Brücken  secondary       50    True   
                    0    Otto-Hirsch-Brücken  secondary       50    True   
                    0    Otto-Hirsch-Brücken  secondary       50    True   
                    0    Otto-Hirsch-Brücken  secondary       50    True   
                    0    Otto-Hirsch-Brücken  secondary       50    True   

                              reversed   length  \
u         v         key                           
279568483 287347230 0    [False, True]  298.065   
287347230 279568483 0    [False, True]  298.065   
                    0    [False, True]  298.065   
                    0    [False, True]  298.065   
                    0    [False, True]  298.065   
...                                ...      ...   
373389416 373252315 0            False   71.894   
                    0            False   71.894   
                    0            False   71.894   
                    0            False   71.894   
                    0            False   71.894   

                                                                  geometry  \
u         v         key                                                      
279568483 287347230 0    LINESTRING (516968.962 5398444.913, 516867.847...   
287347230 279568483 0    LINESTRING (516690.066 5398551.535, 516702.561...   
                    0    LINESTRING (516690.066 5398551.535, 516702.561...   
                    0    LINESTRING (516690.066 5398551.535, 516702.561...   
                    0    LINESTRING (516690.066 5398551.535, 516702.561...   
...                                                                    ...   
373389416 373252315 0    LINESTRING (518737.569 5400767.631, 518754.906...   
                    0    LINESTRING (518737.569 5400767.631, 518754.906...   
                    0    LINESTRING (518737.569 5400767.631, 518754.906...   
                    0    LINESTRING (518737.569 5400767.631, 518754.906...   
                    0    LINESTRING (518737.569 5400767.631, 518754.906...   

                         ...  \
u         v         key  ...   
279568483 287347230 0    ...   
287347230 279568483 0    ...   
                    0    ...   
                    0    ...   
                    0    ...   
...                      ...   
373389416 373252315 0    ...   
                    0    ...   
                    0    ...   
                  

# Visualize Result 

In [9]:
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = gdf_utm['geometry'].to_frame()
temp = point_locs.plot(ax = ax)

# this code plot all the road system 
edges_utm.plot(ax=ax)

# matched point plot
matched_point_df.loc[matched_point_df['method'] == 'IMP'].plot(ax = ax, color = "Green")
# matched point plot
matched_point_df.loc[matched_point_df['method'] == 'SMP1'].plot(ax = ax, color = "Red")
# matched point plot
matched_point_df.loc[matched_point_df['method'] == 'SMP2'].plot(ax = ax, color = "Yellow")


# matched_edge 
matched_edge_df.loc[matched_edge_df['connectivity'] == 0 ].plot(ax = ax, color = "Orange")
matched_edge_df.plot(ax = ax, color = "Black")


<Axes: >

In [17]:
matched_point_list

[                         geometry                   lon_lat method
 0  POINT (516809.007 5398510.072)  POINT (9.22862 48.73938)    IMP,
                          geometry                   lon_lat method
 0  POINT (516788.872 5398517.692)  POINT (9.22835 48.73945)   SMP2,
                          geometry                   lon_lat method
 0  POINT (516786.066 5398536.915)  POINT (9.22831 48.73962)   SMP2,
                          geometry                   lon_lat method
 0  POINT (516762.623 5398546.316)  POINT (9.22799 48.73971)   SMP2,
                          geometry                   lon_lat method
 0  POINT (516728.869 5398559.854)  POINT (9.22753 48.73983)   SMP2,
                          geometry                   lon_lat method
 0  POINT (516719.496 5398554.635)  POINT (9.22741 48.73979)   SMP2,
                          geometry                   lon_lat method
 0  POINT (516707.743 5398568.273)  POINT (9.22725 48.73991)   SMP2,
                          geometry       